In [1]:
import os 
cwd = os.getcwd()
cwd + "\\test.html"

'e:\\rdv_finder\\test.html'

## Methods dev chunk

In [ ]:
import time
import random
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from PIL import Image

def setup_driver():
    user_agents = [
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.82 Safari/537.36",
        "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Firefox/89.0",
    ]
    window_sizes = ["1920x1080", "1366x768", "1280x720"]

    options = Options()
    options.add_argument(f'user-agent={random.choice(user_agents)}')
    options.add_argument(f'--window-size={random.choice(window_sizes)}')
    # options.add_argument('--headless')  # Run in headless mode if required

    driver = webdriver.Chrome(options=options)
    return driver

def get_html(url):
    driver = setup_driver()
    try:
        driver.get(url)
        # Wait 10 seconds so that the content can be fully loaded
        wait = WebDriverWait(driver, 20)
        wait.until(lambda driver: driver.execute_script('return document.readyState') == 'complete')
        # Mimicking human behavior by waiting a random time
        time.sleep(random.randint(2,5))
        html = driver.page_source

    finally:
        driver.quit()
    return html

def get_captcha(html, id):
    soup = BeautifulSoup(html, 'html.parser')
    captcha_element = soup.find(id=id)
    pretty_html = soup.prettify()
    captcha_url = captcha_element['src']
    captcha_url = captcha_url[5:]
    image_response = requests.get(captcha_url)
    if image_response.status_code == 200:
        with open("downloaded_image.png", "wb") as file:
            file.write(image_response.content)
        print("Image saved as downloaded_image.png")
    else:
        print("Failed to download the image")
    # Specify the path and name of your HTML file
    file_path = cwd + '\\test.png'
    # Open a file at the specified path in 'write' mode
    with open(file_path, 'w', encoding='utf-8') as file:
        file.write(pretty_html)  # Write the JavaScript code to the file




def get_captcha(url):
    cwd = os.getcwd()
    screenshot_path = cwd + "\\full_screenshot.png"
    driver = setup_driver()
    try:
        driver.get(url)
        # Wait 10 seconds so that the content can be fully loaded
        wait = WebDriverWait(driver, 20)
        wait.until(lambda driver: driver.execute_script('return document.readyState') == 'complete')
        # Mimicking human behavior by waiting a random time
        time.sleep(random.randint(2,5))
        # Scroll to a specific position (e.g., 500 pixels down)
        scroll_position = 1000  # Replace with the desired vertical scroll position in pixels
        driver.execute_script(f"window.scrollTo(0, {scroll_position});")
        time.sleep(2)  # Adjust sleep time if needed for page loading
        driver.save_screenshot("full_screenshot.png")
        with Image.open(screenshot_path) as img:
            width, height = img.size
    finally:
        driver.quit()
    return width, height

In [37]:
url = 'https://www.rdv-prefecture.interieur.gouv.fr/rdvpref/reservation/demarche/4443/cgu/'
cwd = os.getcwd()
screenshot_path = cwd + "\\full_screenshot.png"
driver = setup_driver()
try:
    driver.get(url)
    # Wait 10 seconds so that the content can be fully loaded
    wait = WebDriverWait(driver, 20)
    wait.until(lambda driver: driver.execute_script('return document.readyState') == 'complete')
    # Mimicking human behavior by waiting a random time
    time.sleep(random.randint(2,5))
    # Scroll to a specific position (e.g., 500 pixels down)
    captcha_image = driver.find_element(By.ID, "captchaFR_CaptchaImage")
    captcha_image.screenshot("captcha.png")

    # scroll_position = 1000  # Replace with the desired vertical scroll position in pixels
    # driver.execute_script(f"window.scrollTo(0, {scroll_position});")
    # time.sleep(2)  # Adjust sleep time if needed for page loading
    # driver.save_screenshot("full_screenshot.png")
    # with Image.open(screenshot_path) as img:
    #     width, height = img.size
finally:
    driver.quit()

In [ ]:
# Test get_captcha function
url = 'https://www.rdv-prefecture.interieur.gouv.fr/rdvpref/reservation/demarche/4443/cgu/'

width, height = get_captcha(url)

cw, ch = width/2, height/2
r = 0.15

with Image.open("full_screenshot.png") as full_img:
    fine_crop = 140 # Fine-tune the cropping by adjusting this value
    fine_crop_right = 100 # Fine-tune the cropping by adjusting this value
    # (left, top, right, bottom)
    element_screenshot = full_img.crop((cw-r*width, ch-r*height+fine_crop, cw+r*width-fine_crop_right, ch+r*height-fine_crop))
    element_screenshot.save("element_screenshot.png")  # Save the cropped image

print("JavaScript file written successfully!")

## Get captchas loop

In [ ]:
num_captchas = 2
for i in range(num_captchas):
    url = 'https://www.rdv-prefecture.interieur.gouv.fr/rdvpref/reservation/demarche/4443/cgu/'

    width, height = get_captcha(url)
    cw, ch = width/2, height/2
    r = 0.2

    with Image.open("full_screenshot.png") as full_img:
        fine_crop = 120 # Fine-tune the cropping by adjusting this value
        fine_crop_right = 100 # Fine-tune the cropping by adjusting this value
        # (left, top, right, bottom)
        element_screenshot = full_img.crop((cw-r*width, ch-r*height, cw+r*width-fine_crop_right, ch+r*height))
        element_screenshot.save(f"captchas/screenshot_{i}.png")  # Save the cropped image

    print(f"JavaScript file written successfully {i+1} times!")

JavaScript file written successfully 1 times!
JavaScript file written successfully 2 times!


In [26]:
r = 0.2
with Image.open("full_screenshot.png") as full_img:
        fine_crop = 120 # Fine-tune the cropping by adjusting this value
        fine_crop_right = 100 # Fine-tune the cropping by adjusting this value
        # (left, top, right, bottom)
        element_screenshot = full_img.crop((cw-r*width+30, ch-r*height+110, cw+r*width-80, ch+r*height-150))
        element_screenshot.save(f"screenshot.png")  # Save the cropped image

## ViT Model

In [5]:
from transformers import DonutProcessor, VisionEncoderDecoderModel, AutoTokenizer, AutoModelForSeq2SeqLM
import torch
from PIL import Image

# Load Donut model and processor
model_name = "naver-clova-ix/donut-base-finetuned-docvqa"  # or another fine-tuned model
processor = DonutProcessor.from_pretrained(model_name,
                                           cache_dir="./cache",  # Specify a cache directory
                                           force_download=False,  # Force re-download of the model files
                                           local_files_only=True)  # Ensure it tries to download if not available locally)
model = VisionEncoderDecoderModel.from_pretrained(model_name)

# Move model to device (CPU or GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

e:\rdv_finder\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


OSError: We couldn't connect to 'https://huggingface.co' to load this file, couldn't find it in the cached files and it looks like naver-clova-ix/donut-base-finetuned-docvqa is not the path to a directory containing a file named preprocessor_config.json.
Checkout your internet connection or see how to run the library in offline mode at 'https://huggingface.co/docs/transformers/installation#offline-mode'.

In [18]:
# Load an image from file
#image_path = "captchas/screenshot_1.png"
image_path = "captchas/simple.jpg"
image = Image.open(image_path).convert("RGB")

# Preprocess the image for the model
pixel_values = processor(images=image, return_tensors="pt").pixel_values
pixel_values = pixel_values.to(device)
pixel_values.shape

torch.Size([1, 3, 2560, 1920])

In [23]:
# Generate text
with torch.no_grad():
    outputs = model.generate(pixel_values, max_length=8, min_length=6, num_beams=5)

# Decode the generated text
recognized_text = processor.decode(outputs[0], skip_special_tokens=True)

print(f"Recognized Text: {recognized_text}")


Recognized Text: m m m m m m


### This actually works very well!!!

In [6]:
from transformers import TrOCRProcessor, VisionEncoderDecoderModel
from PIL import Image
import torch

# Load the pre-trained TrOCR model and processor
processor = TrOCRProcessor.from_pretrained("microsoft/trocr-base-printed")
model = VisionEncoderDecoderModel.from_pretrained("microsoft/trocr-base-printed")

e:\rdv_finder\venv\Lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\march\.cache\huggingface\hub\models--microsoft--trocr-base-printed. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Config of the encoder: <class 'transformers.models.vit.modeling_vit.ViTModel'> is overwritten by shared encoder config: V

In [29]:
# Load and preprocess the image
image = Image.open("captchas/screenshot_1.png").convert("RGB")
pixel_values = processor(images=image, return_tensors="pt").pixel_values

# Perform OCR
outputs = model.generate(pixel_values)
recognized_text = processor.batch_decode(outputs, skip_special_tokens=True)[0]
print("Recognized Text:", recognized_text)

Recognized Text: PUSKIT


## Putting it all together

In [45]:
import time
import random
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from transformers import TrOCRProcessor, VisionEncoderDecoderModel
from PIL import Image
import torch
import re

# Load the processor and model
processor = TrOCRProcessor.from_pretrained("microsoft/trocr-base-printed")
model = VisionEncoderDecoderModel.from_pretrained("microsoft/trocr-base-printed")

# # Get the tokenizer
# tokenizer = processor.tokenizer

# # Modify the vocabulary to include only numbers and letters
# allowed_tokens = [token for token in tokenizer.get_vocab() if token.isalnum()]
# tokenizer.add_tokens(allowed_tokens)

# # Set the new tokenizer back to the processor
# processor.tokenizer = tokenizer

# Set the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Config of the encoder: <class 'transformers.models.vit.modeling_vit.ViTModel'> is overwritten by shared encoder config: ViTConfig {
  "attention_probs_dropout_prob": 0.0,
  "encoder_stride": 16,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 768,
  "image_size": 384,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "model_type": "vit",
  "num_attention_heads": 12,
  "num_channels": 3,
  "num_hidden_layers": 12,
  "patch_size": 16,
  "qkv_bias": false,
  "transformers_version": "4.46.0"
}

Config of the decoder: <class 'transformers.models.trocr.modeling_trocr.TrOCRForCausalLM'> is overwritten by shared decoder config: TrOCRConfig {
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "add_cross_attention": true,
  "attention_dropout": 0.0,
  "bos_token_id": 0,
  "classifier_dropout": 0.0,
  "cross_attention_hidden_size": 768,
  "d_model": 1024,
  "decoder_attention_heads": 16,
  "decoder_ffn_dim": 4096,
  "decoder

In [60]:
import cv2
import numpy as np
for i in range(1, 21):
    image_pathe = f"captchas/screenshot_{i}.png"
    image = cv2.imread(image_pathe, cv2.IMREAD_GRAYSCALE)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    image = clahe.apply(image)
    kernel_size = [2, 2]
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, kernel_size)
    cleaned = cv2.morphologyEx(image, cv2.MORPH_CLOSE, kernel)
    cv2.imwrite(f"captchas/cleaned_{i}.png", image)

In [61]:
max_attempts = 20
timeout = 10
url = 'https://www.rdv-prefecture.interieur.gouv.fr/rdvpref/reservation/demarche/4443/cgu/'
cwd = os.getcwd()
screenshot_path = cwd + "\\full_screenshot.png"
driver = setup_driver()
try:
    count = 0
    driver.get(url)
    # Wait 10 seconds so that the content can be fully loaded
    wait = WebDriverWait(driver, timeout)
    wait.until(lambda driver: driver.execute_script('return document.readyState') == 'complete')
    # Mimicking human behavior by waiting a random time
    time.sleep(random.randint(2,5))
    # Scroll to a specific position (e.g., 500 pixels down)
    # scroll_position = 1000  # Replace with the desired vertical scroll position in pixels
    # driver.execute_script(f"window.scrollTo(0, {scroll_position});")
    time.sleep(2)  # Adjust sleep time if needed for page loading
    current_url = driver.current_url
    expected_url = "https://www.rdv-prefecture.interieur.gouv.fr/rdvpref/reservation/demarche/4443/creneau/"
    while current_url != expected_url and count < max_attempts:
        # driver.save_screenshot("full_screenshot.png")
        # with Image.open(screenshot_path) as img:
        #     width, height = img.size
        # cw, ch = width/2, height/2
        # r = 0.2
        # with Image.open("full_screenshot.png") as full_img:
        #     # (left, top, right, bottom)
        #     coords = (cw-r*width+30, ch-r*height+110, cw+r*width-80, ch+r*height-150)
        #     element_screenshot = full_img.crop(coords)
        #     element_screenshot.save(f"captchas/screenshot_{count+1}.png")  # Save the cropped image
        image_path = f"captchas/screenshot_{count+1}.png"
        captcha_image = WebDriverWait(driver, timeout).until(
            EC.presence_of_element_located((By.ID, "captchaFR_CaptchaImage"))
        )
        captcha_image = driver.find_element(By.ID, "captchaFR_CaptchaImage")
        captcha_image.screenshot(image_path)
        ###########################################
        captcha_image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
        captcha_image = clahe.apply(captcha_image)
        kernel_size = [2, 2]
        kernel = cv2.getStructuringElement(cv2.MORPH_RECT, kernel_size)
        captcha_image = cv2.morphologyEx(captcha_image, cv2.MORPH_CLOSE, kernel)
        cv2.imwrite(f"captchas/cleaned_{i}.png", captcha_image)
        ###########################################
        cleaned_path = f"captchas/cleaned_{count+1}.png"
        image = Image.open(cleaned_path).convert("RGB")
        pixel_values = processor(images=image, return_tensors="pt").pixel_values
        pixel_values = pixel_values.to(device)
        with torch.no_grad():
            outputs = model.generate(pixel_values, max_length=9, min_length=6, num_beams=5)
        decoded_text = processor.decode(outputs[0], skip_special_tokens=True)
        recognized_text = re.sub(r"[^A-Z0-9]", "", decoded_text)
        print(f"Recognized Text {count+1}: {recognized_text}")
        captcha_input = driver.find_element(By.ID, "captchaFormulaireExtInput")
        captcha_input.send_keys(recognized_text)
        button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//button[.//span[text()='Suivant']]"))
        )
        button.click()
        # Wait until the next page is loaded
        wait.until(lambda driver: driver.execute_script('return document.readyState') == 'complete')
        current_url = driver.current_url
        count += 1
    if count >= max_attempts:
        print("Failed to pass the CAPTCHA!")
    else:
        print("Successfully passed the CAPTCHA!")
    # Wait for the button to be present in the DOM
    button = WebDriverWait(driver, timeout).until(
        EC.presence_of_element_located((By.CLASS_NAME, "q-btn--actionable"))  # Replace with the correct locator
    )
    button.click()  # Perform click action
    expected_url = "https://www.rdv-prefecture.interieur.gouv.fr/rdvpref/reservation/demarche/4443/confirmation/"
    current_url = driver.current_url
    if current_url == expected_url:
        print("Successfully entered the confirmation page!")
    else:
        print("Failed to enter the confirmation page!")
finally:
    driver.quit()

# cw, ch = width/2, height/2
# r = 0.2

# with Image.open("full_screenshot.png") as full_img:
#     # (left, top, right, bottom)
#     coords = (cw-r*width+30, ch-r*height+110, cw+r*width-80, ch+r*height-150)
#     element_screenshot = full_img.crop(coords)
#     element_screenshot.save(f"captchas/screenshot_{i+1}.png")  # Save the cropped image
# image_path = f"captchas/screenshot_{i+1}.png"
# image = Image.open(image_path).convert("RGB")
# pixel_values = processor(images=image, return_tensors="pt").pixel_values
# pixel_values = pixel_values.to(device)

# with torch.no_grad():
#     outputs = model.generate(pixel_values, max_length=8, min_length=6, num_beams=5)

# recognized_text = processor.decode(outputs[0], skip_special_tokens=True)
# print(f"Recognized Text {i+1}: {recognized_text}")

# print(f"JavaScript file written {i+1} times!")

Recognized Text 1: NSA3VE
Recognized Text 2: X40N6X
Recognized Text 3: UM9FK
Recognized Text 4: 06514
Recognized Text 5: 9STURK
Recognized Text 6: DULEN431
Recognized Text 7: SYCASHED
Recognized Text 8: 4E09MM3
Recognized Text 9: S165MMUM
Recognized Text 10: TW406
Recognized Text 11: BM7M6X
Recognized Text 12: BSBA3N
Recognized Text 13: AAXANS
Recognized Text 14: 46SXNKY
Recognized Text 15: T159KAP53
Recognized Text 16: AVIS35X
Recognized Text 17: 83984RA6
Recognized Text 18: 3STRSHT3
Recognized Text 19: M995MM
Recognized Text 20: 54MMBXB
Failed to pass the CAPTCHA!
Failed to enter the confirmation page!
